In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'amazon-fine-food-reviews:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F18%2F2157%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240422%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240422T220253Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db8c205729ef930a3fb01702979da61ce7c4f46f8e4c0e2f6e6417cbc26ca2027c445791f39505badfaace6c3a2975b046d4ce171062230b61abf7a97dae1bb6f3c48c2c79d0ddd05413680860775b1a14c510c82d683ce0a897293343944b9ba52b4ead2b3451eb248b0ad3bbcea3d53c26b5e7c7f578483c71abd29b96ff3caebe9da41ee2ab07b7bc56edcb3a01c9940e94611a74945af88430e8ff38f148c0edb0900aa209c4d6e140eef3e29532dd1f7d83d48cc363e5e2807a116331004fc12df7fb9f057f56cc76b180cf1c70a3f877905d7902772c7369ddeb5a235f503aba924f5beae30a0e9d8d85c13edb846de3bf70c843b4a933cd09e7a7c11d1,preprocessedd:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4855174%2F8196682%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240422%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240422T220253Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6467c152f4cd622c8b188ebf7cd8933ba3b3553cc974a13c576b6fc464f19690d9b0cedda5cf2a531cf1cb1d8613052abb49a65e0efb4c32b903b84fb245f62e455c95b143b657fed7684d03a6f3cb85f959ce5d3d8ef97965ccc92584e4cf2d8ee865fbf7a5cd81d7988adb9cabfdb5a9b0d0fa833e81fc0e7fc76e7c8a74910b29e619d857dc203961bedc298a29d79d84ec217ba8af60d4dc72df24b426c42eb872873f350f209255ffe3e615c64a85f7a1ab9bf7f3595572370b304d9b683058b8af42c07e70b498f3f2b6166b875e2b405e48e6285d6ea34d12ab61b5613fd5287dd68b16435e658686feb83697398f9c671c08917ce5f80112290fb1bd'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import torch
import pandas as pd
import os
import numpy as np
import nltk
import re
import string
import pickle
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
import math
from sklearn.preprocessing import StandardScaler
from bs4 import BeautifulSoup
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning, module="transformers")

# Download NLTK stopwords if not already downloaded
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df = pd.read_csv("/kaggle/input/amazon-fine-food-reviews/Reviews.csv")[:40000]

df.head

<bound method NDFrame.head of           Id   ProductId          UserId                      ProfileName  \
0          1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1          2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2          3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3          4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4          5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   
...      ...         ...             ...                              ...   
39995  39996  B00017LEY8  A2V2FL4XWQ3TU4                              kes   
39996  39997  B00017LEY8   AHNWS8JUU0D48       Jack Parker "photographer"   
39997  39998  B00017LEY8  A36G9KA8FGCVS1                       John ONeal   
39998  39999  B00017LEY8  A1DCZ61SUGL12H                          Shayrul   
39999  40000  B00017LEY8    ANVO82YEQNCD                          Russ K.   

       HelpfulnessNumerator  HelpfulnessDenom

In [ ]:
columns = ["Id","ProductId","UserId","ProfileName","HelpfulnessNumerator","HelpfulnessDenominator","Score","Time"]

df = df.drop(columns=columns)

In [ ]:



from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

def lowercase_text(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text().lower()

def remove_stopwords(tokens):
    return [word for word in tokens if word not in stopwords.words('english')]

def preprocess_text_pipeline(text):
    # Check if the text is a string
    if not isinstance(text, str):
        return ''  # Return an empty string for non-string inputs

    text = lowercase_text(text)
    tokens = word_tokenize(text)
    tokens = remove_stopwords(tokens)
    return ' '.join(tokens)


In [ ]:

df['Text'] = df['Text'].apply(preprocess_text_pipeline)

df = df.dropna(subset=['Text', 'Summary'])

df.head()

,Summary,Text
0,Good Quality Dog Food,bought several vitality canned dog food produc...
1,Not as Advertised,product arrived labeled jumbo salted peanuts ....
2,"""Delight"" says it all","confection around centuries . light , pillowy ..."
3,Cough Medicine,looking secret ingredient robitussin believe f...
4,Great taffy,great taffy great price . wide assortment yumm...


In [ ]:

# Save DataFrame to a pickle file
pickle_file_path = 'preprocessed_data.pkl'
df.to_pickle(pickle_file_path)

# Check the first few rows of the DataFrame
print(df.head())


                 Summary                                               Text
0  Good Quality Dog Food  bought several vitality canned dog food produc...
1      Not as Advertised  product arrived labeled jumbo salted peanuts ....
2  "Delight" says it all  confection around centuries . light , pillowy ...
3         Cough Medicine  looking secret ingredient robitussin believe f...
4            Great taffy  great taffy great price . wide assortment yumm...


In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_length=70):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.sep_token = "TL;DR"  # Set TL;DR as the separator token

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = self.data.iloc[index]['Text']
        summary = self.data.iloc[index]['Summary']

        # Add TL;DR separator between text and summary
        concatenated_text = f"{text} {self.sep_token} {summary}"

        inputs = self.tokenizer.encode_plus(
            concatenated_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
        )

        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': summary
        }

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Initialize the GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
train_dataset = CustomDataset(train_df, tokenizer)
test_dataset = CustomDataset(test_df, tokenizer)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 45

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
# Print the number of training and testing samples
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")

Number of training samples: 29999
Number of testing samples: 10000


In [ ]:
# Initialize GPT-2 model and tokenizer
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW, get_linear_schedule_with_warmup

model = GPT2LMHeadModel.from_pretrained('gpt2')  # Load pre-trained GPT-2 model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')  # Load GPT-2 tokenizer

# Prepare your dataset and create data loaders (assuming you have already done this)

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * 10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(10):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = input_ids.clone()  # Use input_ids as labels for auto-regressive training

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    # Calculate average loss for the epoch
    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{10}, Average Loss: {avg_loss:.4f}')

Epoch 1/10, Average Loss: 3.5189
Epoch 2/10, Average Loss: 3.2645
Epoch 3/10, Average Loss: 3.1629
Epoch 4/10, Average Loss: 3.0902
Epoch 5/10, Average Loss: 3.0347
Epoch 6/10, Average Loss: 2.9903
Epoch 7/10, Average Loss: 2.9566
Epoch 8/10, Average Loss: 2.9284
Epoch 9/10, Average Loss: 2.9077
Epoch 10/10, Average Loss: 2.8942


In [ ]:
# Define the path where you want to save the model
save_path = "/kaggle/working/model.pt"

# Save the model's state dictionary
torch.save(model.state_dict(), save_path)

print(f"Model saved to {save_path}")

Model saved to /kaggle/working/model.pt


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=6c00110503cfd17e67e5cf88292d171e90e3288d8ce31cf23fa84d5bf7efc51a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_metric
from tqdm import tqdm

# Define the device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the path where the model is saved
load_path = "/kaggle/working/model.pt"

# Initialize a new model instance and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)  # Move model to GPU
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load the state dictionary from the saved file
model.load_state_dict(torch.load(load_path, map_location=device))

# Load ROUGE metric from Hugging Face datasets library
rouge_metric = load_metric('rouge')

# Evaluate loop
model.eval()
generated_summaries = []
original_summaries = []

for batch in tqdm(test_loader, desc="Evaluating"):
    input_ids = batch['input_ids'].to(device)  # Move input tensors to GPU
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['label']

    # Generate summaries using the model
    with torch.no_grad():
        outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=100, num_beams=1, early_stopping=True)

    # Decode generated summaries and original summaries
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    original_text = labels[0]

    # Extract the summary part after the TL;DR token
    generated_summary = generated_text.split('TL;DR')[1].strip() if 'TL;DR' in generated_text else generated_text
    original_summary = original_text.split('TL;DR')[1].strip() if 'TL;DR' in original_text else original_text

    # Append to lists for ROUGE computation
    generated_summaries.append(generated_summary)
    original_summaries.append(original_summary)

    # Print generated and original summaries
    print(f"Generated Summary: {generated_summary}")
    print(f"Original Summary: {original_summary}")
    print("-" * 50)

# Compute ROUGE scores for evaluation
rouge_scores = rouge_metric.compute(predictions=generated_summaries, references=original_summaries)
print("ROUGE Scores:")
print(rouge_scores)

Evaluating:   0%|          | 0/223 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Evaluating:   0%|          | 1/223 [00:01<06:36,  1.78s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Generated Summary: VERY HAPPY WITH THIS CAYENNE!!!
Original Summary: VERY HAPPY WITH THIS CAYENNE!!!
--------------------------------------------------


Evaluating:   1%|          | 2/223 [00:02<03:51,  1.05s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Generated Summary: bought mother recommendation family physician. said mom iron deficiency anemia. name implies, iron deficiency anemia due insufficient iron. without enough iron, body can't produce enough hemoglobin, substance red blood cells enables carry oxygen. result, iron deficiency anemia leave tired short breath. solution : boost iron intake.one tablespoon tasty delight 70 % rda adult recommended dietary allowance ( rda ) recommended dietary allowance ( rda ) amounts selected nutrients considered adequate meet known nutrient needs healthy people. rda
Original Summary: Very High in Iron
--------------------------------------------------


Evaluating:   1%|▏         | 3/223 [00:02<02:58,  1.23it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

Evaluating: 100%|█████████▉| 222/223 [01:57<00:00,  1.90it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Generated Summary: Best instant noodle cup
Original Summary: Best instant noodle cup
--------------------------------------------------


Evaluating: 100%|██████████| 223/223 [01:59<00:00,  1.87it/s]

Generated Summary: The best you can buy
Original Summary: The best you can buy
--------------------------------------------------


ROUGE Scores:
{'rouge1': AggregateScore(low=Score(precision=0.668313244539601, recall=0.6832897318799785, fmeasure=0.6626339957311078), mid=Score(precision=0.7261499587187237, recall=0.7391533852296184, fmeasure=0.7205977065724865), high=Score(precision=0.7788988955336325, recall=0.7922795621170061, fmeasure=0.7736035102749244)), 'rouge2': AggregateScore(low=Score(precision=0.5816602754625324, recall=0.585803437967115, fmeasure=0.580197171958572), mid=Score(precision=0.6444912430206702, recall=0.647089513625388, fmeasure=0.6432555318333792), high=Score(precision=0.7017348658532969, recall=0.7053851184316432, fmeasure=0.7018065230653566)), 'rougeL': AggregateScore(low=Score(precision=0.6674755815422907, recall=0.6830311685076258, fmeasure=0.662376669217374), mid=Score(precision=0.7220670630067134, recall=0.7353072687153404, fmeasure=0.7176696758755234), high=Score(precision=0.7795360345813868, recall=0.7908160237868755, fmeasure=0.7750093718405009)), 'rougeLsum': AggregateScore(low=Scor

In [ ]:
# calculion of Rogue Score
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_metric
from tqdm import tqdm
import pandas as pd
from IPython.display import clear_output

# Define the device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the path where the model is saved
load_path = "/kaggle/input/modelllls/model.pt"

# Initialize a new model instance and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)  # Move model to GPU
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load the state dictionary from the saved file
model.load_state_dict(torch.load(load_path, map_location=device))

# Load ROUGE metric from Hugging Face datasets library
rouge_metric = load_metric('rouge')

# Initialize results data list
results_data = []
i = 1

for example in tqdm(test_dataset, desc="Evaluating"):
    input_ids = example['input_ids'].to(device)
    attention_mask = example['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=100,
            num_beams=1,
            early_stopping=True,
        )
    predicted_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    actual_text = example['text']
    actual_summary = example['summary']
    summary_sentences = [s.strip() for s in predicted_summary.split('.') if s.strip()]
    rouge_scores = []

    print("-------------------------------------------------------------")
    print("Text: " + actual_text)
    print("-------------------------------------------------------------")
    print("Given Summary: " + actual_summary)
    print("-------------------------------------------------------------")

    for sentence in summary_sentences:
        rouge_scores_sentence = rouge_metric.compute(predictions=[sentence], references=[actual_summary], use_stemmer=True)
        rouge_scores.append(rouge_scores_sentence)
        print(f"Sentence: {sentence}")
        for metric_name in rouge_scores_sentence.keys():
            precision = rouge_scores_sentence[metric_name].mid.precision
            recall = rouge_scores_sentence[metric_name].mid.recall
            f1_score = rouge_scores_sentence[metric_name].mid.fmeasure
            score_str = f"{metric_name.upper()}: Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1_score:.2f}"
            print(score_str)
        print("-" * 50)

    results_data.append([actual_text, actual_summary, summary_sentences, rouge_scores])
    print("Row: " + str(i) + "/" + str(len(test_dataset)))
    if i % 100 == 0:
        clear_output(wait=True)
    i += 1

# Create a DataFrame for results
df_results = pd.DataFrame(results_data, columns=['Text', 'Given Summary', 'Generated Sentences', 'ROUGE Scores'])


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Define the device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the path where the model is saved
load_path = "/kaggle/working/model.pt"

# Initialize a new model instance and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)  # Move model to GPU
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load the state dictionary from the saved file
model.load_state_dict(torch.load(load_path, map_location=device))

# Assuming input_text is your input text string
input_text = "you can buy the best things  TL;DR"

# Tokenize the input text and move to GPU
inputs = tokenizer(input_text, return_tensors='pt').to(device)

# Generate output text using the fine-tuned GPT-2 model
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=150,  # Set the maximum length of the generated text
        num_beams=5,  # Adjust the beam search parameters if needed
        early_stopping=True,  # Enable early stopping to finish generation when model is confident
    )

# Decode the generated output tokens back to text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extract the summary part after the TL;DR token
summary = generated_text.split('TL;DR')[1].strip() if 'TL;DR' in generated_text else generated_text
print("Generated Summary:", summary)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Summary: The best


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Define the device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the path where the model is saved
load_path = "/kaggle/working/model.pt"

# Initialize a new model instance and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)  # Move model to GPU
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load the state dictionary from the saved file
model.load_state_dict(torch.load(load_path, map_location=device))

# Assuming input_text is your input text string
input_text = "you can buy the best things TL;DR"

# Tokenize the input text and move to GPU
inputs = tokenizer(input_text, return_tensors='pt').to(device)

# Generate output text using the fine-tuned GPT-2 model
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=150,  # Set the maximum length of the generated text
        num_beams=5,  # Adjust the beam search parameters if needed
        early_stopping=True,  # Enable early stopping to finish generation when model is confident
    )

# Decode the generated output tokens back to text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extract the summary part after the TL;DR token
generated_summary = generated_text.split('TL;DR')[1].strip() if 'TL;DR' in generated_text else generated_text
print("Generated Summary:", generated_summary)

# Define reference summary
reference_summary = "The best things are available for purchase."

# Calculate BLEU score as a measure of similarity
bleu_score = sentence_bleu([reference_summary.split()], generated_summary.split(), smoothing_function=SmoothingFunction().method1)
print("BLEU Score:", bleu_score)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Summary: The best
BLEU Score: 0.025957555738330737
